In [1]:
import pulp as pl
import numpy as np
import pandas as pd

In [2]:
df_distance = pd.read_csv("D:distance_matrix.csv", index_col=0)

In [3]:
distance_matrix = df_distance.to_numpy()

In [4]:
demands =  [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]

# Vehicle capacity
vehicle_capacity = 15
num_vehicles = 4
num_locations = len(demands)

# Depot index
depot = 0

In [6]:
model = pl.LpProblem("Vehicle_Routing_Problem", pl.LpMinimize)

# Create the decision variables
x = pl.LpVariable.dicts("x", 
                        [(i, j, k) for i in range(num_locations) 
                                   for j in range(num_locations) 
                                   for k in range(num_vehicles)], 
                        cat=pl.LpBinary)

# Variable for load carried by each vehicle
load = pl.LpVariable.dicts("load", 
                           [(j, k) for j in range(num_locations) 
                                   for k in range(num_vehicles)], 
                           lowBound=0, cat=pl.LpInteger)


In [7]:
# Minimize the total distance
model += pl.lpSum(
    distance_matrix[i][j] * x[i, j, k]
    for i in range(num_locations)
    for j in range(num_locations)
    for k in range(num_vehicles)
)

In [8]:
for j in range(1, num_locations):  # Skip depot
    model += pl.lpSum(x[i, j, k] for i in range(num_locations) for k in range(num_vehicles)) == 1

In [9]:
for k in range(num_vehicles):
    model += pl.lpSum(x[depot, j, k] for j in range(1, num_locations)) == 1  # Start at depot
    model += pl.lpSum(x[j, depot, k] for j in range(1, num_locations)) == 1  # Return to depot

In [10]:
for k in range(num_vehicles):
    for j in range(1, num_locations):
        model += load[j, k] <= vehicle_capacity

# Ensure load consistency when a vehicle moves from node i to node j
for k in range(num_vehicles):
    for i in range(num_locations):
        for j in range(1, num_locations):
            if i != j:
                model += load[j, k] >= load[i, k] + demands[j] - (1 - x[i, j, k]) * vehicle_capacity


In [11]:
for k in range(num_vehicles):
    for i in range(num_locations):
        model += x[i, i, k] == 0

In [12]:
# Solve the VRP
model.solve()

# Check the solution status
if pl.LpStatus[model.status] == 'Optimal':
    print("\nOptimal Solution Found!\n")
else:
    print("\nNo Optimal Solution Found\n")


Optimal Solution Found!



In [13]:
# Display the route for each vehicle
total_distance = 0

for k in range(num_vehicles):
    print(f"\nRoute for Vehicle {k + 1}:")
    
    route = []
    route_distance = 0
    current_location = depot

    # Track visited nodes to avoid infinite loops
    visited = set()

    while True:
        next_location = None

        # Iterate over all possible destinations
        for j in range(num_locations):
            if current_location != j and pl.value(x[current_location, j, k]) == 1:
                route.append(j)
                route_distance += distance_matrix[current_location][j]
                visited.add(current_location)
                current_location = j
                next_location = j
                break

        # Check if the vehicle returns to the depot or there are no further moves
        if next_location is None or current_location == depot or len(visited) == num_locations - 1:
            break

    total_distance += route_distance

    # Handle vehicles with no assigned routes
    if len(route) == 0:
        print("No route assigned.")
    else:
        print(f"Route: {route}")
        print(f"Distance: {route_distance} meters")

print(f"\nTotal Distance: {total_distance} meters")



Route for Vehicle 1:
Route: [9, 0]
Distance: 388 meters

Route for Vehicle 2:
Route: [7]
Distance: 194 meters

Route for Vehicle 3:
Route: [5]
Distance: 274 meters

Route for Vehicle 4:
Route: [8]
Distance: 308 meters

Total Distance: 1164 meters
